# Test reading a set of rainfields netCDF rain_rate files from the mongo database

In [28]:
import datetime
import os
import gridfs as fs
from pymongo import MongoClient
from netCDF4 import Dataset
from package import rain_base_fs

In [29]:
config = { 
    "START_DATE" : "2021-01-28T00:00:00",
    "END_DATE" : "2021-01-28T01:00:00",
    "TIME_STEP": "5", 
    "BASE_DIR": "/home/awseed/data/RF3/prcp-crate",
    "PRODUCT": "prcp-crate.nc",
    "STN_ID": "310",
    "DB_NAME": "radar",
    "DROP_DATABASE":"1",
    "VARIABLE":"rain_rate"
}

In [30]:
client = MongoClient()
db = client[config["DB_NAME"]]
radar_fs = fs.GridFS(db)
file_list = radar_fs.list()
file_name = file_list[0] 

In [85]:
# loop over the dates and read in the data
valid_time = datetime.datetime.fromisoformat(config["START_DATE"]).replace(tzinfo= datetime.timezone.utc)
end_time = datetime.datetime.fromisoformat(config["END_DATE"]).replace(tzinfo = datetime.timezone.utc)
time_step = datetime.timedelta(minutes=int(config["TIME_STEP"]))
station_id = int(config["STN_ID"])
variable = config["VARIABLE"] 
filter = {
    "metadata.station_id":station_id,
    "metadata.variable":variable
}
ts = int(datetime.datetime.timestamp(valid_time))
filter["metadata.valid_time"] = ts
print(f"{filter=}")

col = db.fs.files.find_one(filter) 
if col is None:
    print("None found")
else:    
    print(col["metadata"])
    print(col["filename"])

# while valid_time <= end_time:
#     print(int(datetime.datetime.timestamp(valid_time)))
#     ts = int(datetime.datetime.timestamp(valid_time))
#     filter["valid_time"] = ts
#     cursor = radar_fs.find(filter)
#     for grid_out in cursor:
#         print(grid_out)

    # for grid_out in radar_fs.find(filter):
    #     data = grid_out.read()
    #     ncFile = Dataset("fname.nc", mode="r", memory=data) 
    #     station_id = int(ncFile.__getattr__("station_id"))
    #     station_name = str(ncFile.__getattr__("station_name"))
    #     vtime = int(ncFile['valid_time'][0].item())
    #     print(f"{station_id=}, {station_name=},{vtime=}")
    #     ncFile.close()
    # valid_time = valid_time + time_step



filter={'metadata.station_id': 310, 'metadata.variable': 'rain_rate', 'metadata.valid_time': 1611792000}
{'station_id': 310, 'station_name': 'Ausm310', 'valid_time': 1611792000, 'variable': 'rain_rate', 'mean': 0.007201567806456096, 'std': 0.05467740277750921, 'war': 1.9590372330712125}
310_20210128_000000.prcp-crate.nc


In [ ]:
client.close()